# Tracking de Top 5 Usuarios en el Top4 Lugares - Rango 5 minutos

Se realizo la creacion de los archivos de nodos y arcos, por cada lugar, pero se tomaron diferentes rangos de tiempo siendo estos 5, 10 y 15 minutos. Para elegir el mejor escenario para el estudio. En este caso analizaremos el rango de 15 minutos.

# Importaciones

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Carga del Dataset 

In [2]:
conexiones = pd.DataFrame.from_csv("dataset_final.csv", sep = ",")
conexiones.head(3)

,datetime,codigo,pauta_id,sitio_id,mac_usr,ip_local,ip_usr,so,navegador,url_ref,tiempo_espera,fabricante-dispositivo
0,2015-05-15 20:16:35,1,11,2,94:01:C2:97:96:CB,10.5.8.1,10.5.10.95,Android,SAFARI,http://10.5.8.1/status,00:00:00,"Samsung Electronics Co.,Ltd"
1,2015-05-15 20:18:15,2,5,2,8C:BF:A6:17:60:A0,10.5.8.1,10.5.9.24,Android,CHROME,http://10.5.8.1/status,00:00:00,"Samsung Electronics Co.,Ltd"
2,2015-05-15 20:18:24,3,19,3,74:AA:FE:A5:7E:90,10.5.0.1,10.5.0.176,Android,CHROME,http://10.5.0.1/status,00:00:00,NaN


In [3]:
conexiones["datetime"] = pd.to_datetime(conexiones["datetime"])
conexiones["mes"] = conexiones["datetime"].dt.month
conexiones["año"] = conexiones["datetime"].dt.year
conexiones["hora"] = conexiones["datetime"].dt.hour
conexiones["fecha"] = conexiones["datetime"].dt.date
conexiones["weekday"] = conexiones["datetime"].dt.weekday
conexiones.head(2)

,datetime,codigo,pauta_id,sitio_id,mac_usr,ip_local,ip_usr,so,navegador,url_ref,tiempo_espera,fabricante-dispositivo,mes,año,hora,fecha,weekday
0,2015-05-15 20:16:35,1,11,2,94:01:C2:97:96:CB,10.5.8.1,10.5.10.95,Android,SAFARI,http://10.5.8.1/status,00:00:00,"Samsung Electronics Co.,Ltd",5,2015,20,2015-05-15,4
1,2015-05-15 20:18:15,2,5,2,8C:BF:A6:17:60:A0,10.5.8.1,10.5.9.24,Android,CHROME,http://10.5.8.1/status,00:00:00,"Samsung Electronics Co.,Ltd",5,2015,20,2015-05-15,4


In [4]:
conexionesFinal = conexiones[["datetime","sitio_id", "mac_usr", "so", "navegador", "fabricante-dispositivo", "hora", "weekday", "fecha", "mes", "año"]]
conexionesFinal.head(2)

,datetime,sitio_id,mac_usr,so,navegador,fabricante-dispositivo,hora,weekday,fecha,mes,año
0,2015-05-15 20:16:35,2,94:01:C2:97:96:CB,Android,SAFARI,"Samsung Electronics Co.,Ltd",20,4,2015-05-15,5,2015
1,2015-05-15 20:18:15,2,8C:BF:A6:17:60:A0,Android,CHROME,"Samsung Electronics Co.,Ltd",20,4,2015-05-15,5,2015


In [5]:
sitios = pd.DataFrame.from_csv("ma_sitios2.csv", sep = ",").reset_index()
sitios.head(3)

,sitio_id,nombre,siglas,ruc,ciudad,nombre_ubicacion,contacto,direccion,email,telefono,...,transparencia,longitud,latitud,ip,puerto,fecha,usuario_modif,bienvenida,formulario,router
0,1,ALBAN BORJA,ALBAN BORJ,920124541001,Guayaquil,Guayaquil,Fabricio Matamoros,Av. Carlos Julio Arosemena,administracion@albanborja.com,923154211,...,0,-2.168677,-79.916769,186.3.137.95,NaN,2016-09-19 13:28:32,fmatamoros,0,0,Mikrotik
1,2,VILLAGE PLAZA,VILLA,0,Guayaquil,Guayaquil,Gabriela,SAMBORONDON,NaN,NaN,...,0,-2.140406,-79.866376,186.3.144.18,NaN,2016-07-20 17:23:50,fmatamoros,0,0,NaN
2,3,CITY MALL,CITY M.,0,Guayaquil,Guayaquil,Gabriela,Alborada,NaN,NaN,...,0,-2.140951,-79.911774,186.3.153.140,NaN,2015-12-22 10:21:54,fmatamoros,0,0,NaN


In [6]:
places =[["Urdesa", [[-79.915581,-2.178119], [-79.901204,-2.146938]]],
             ["MallSur", [[-79.902062,-2.232042], [-79.894681,-2.226038]]],
             ["CityMall", [[-79.912663,-2.143936], [-79.905968,-2.137932]]],
             ["AlbanBorja", [[-79.918242,-2.170096], [-79.915066,-2.167523]]],
             ["Samborondon", [[-79.874897,-2.160447], [-79.852066,-2.116704]]],
             ["Aeropuerto", [[-79.886227,-2.152728], [-79.880562,-2.146338]]],
             ["Penias", [[-79.8776014239,-2.1855487464], [-79.873009,-2.175392]]],
             ["Quito", [[-78.586922,-0.395161], [-78.274155,0.021973]]],
             ["CentroConvenciones", [[-79.893479,-2.161755], [-79.883652,-2.154894]]],
             ["HiltonColon", [[-79.8998951912,-2.160929558], [-79.8962903023,-2.1576274229]]],
             ["MallSol", [[-79.8954641819,-2.157053837], [-79.8891341686,-2.1539258197]]],
             ["UniversidadGuayaquil", [[-79.9008607864,-2.1847947751], [-79.8959684372,-2.1791554991]]]
            ]

In [7]:
def localizar(row, places):
    longitud = float(row["longitud"])
    latitud = float(row["latitud"])

    for placeName, ubicacion in places:
        SW, NE= ubicacion
        if (latitud <= NE[0] and longitud <=NE[1] and latitud >= SW[0] and longitud >= SW[1]):
            return placeName
    return "No ubicado"

In [8]:
sitios['localizacion'] = sitios[["longitud", "latitud"]].apply(lambda row: localizar(row, places), axis = 1)

In [9]:
sitios = sitios[["sitio_id","nombre", "longitud", "latitud", "localizacion"]]
sitios.head(3)

,sitio_id,nombre,longitud,latitud,localizacion
0,1,ALBAN BORJA,-2.168677,-79.916769,AlbanBorja
1,2,VILLAGE PLAZA,-2.140406,-79.866376,Samborondon
2,3,CITY MALL,-2.140951,-79.911774,CityMall


In [10]:
datos = pd.merge(conexionesFinal, sitios[["sitio_id","nombre", "longitud", "latitud", "localizacion"]], on = "sitio_id")
datos["hora"] = datos["hora"].apply(lambda x: str(x).rjust(2,"0") + "h")
datos.head(3)

,datetime,sitio_id,mac_usr,so,navegador,fabricante-dispositivo,hora,weekday,fecha,mes,año,nombre,longitud,latitud,localizacion
0,2015-05-15 20:16:35,2,94:01:C2:97:96:CB,Android,SAFARI,"Samsung Electronics Co.,Ltd",20h,4,2015-05-15,5,2015,VILLAGE PLAZA,-2.140406,-79.866376,Samborondon
1,2015-05-15 20:18:15,2,8C:BF:A6:17:60:A0,Android,CHROME,"Samsung Electronics Co.,Ltd",20h,4,2015-05-15,5,2015,VILLAGE PLAZA,-2.140406,-79.866376,Samborondon
2,2015-05-15 20:20:20,2,34:BE:00:05:16:18,Android,SAFARI,"Samsung Electronics Co.,Ltd",20h,4,2015-05-15,5,2015,VILLAGE PLAZA,-2.140406,-79.866376,Samborondon


# Analisis de los Lugares 

En base al estudio previo se determinó que el Top 4 de Lugares en nuestro dataset esta formado por:
 - Aeropuerto
 - Mall del Sur
 - City-Mall
 - Centro de Convenciones
 
Se procedera a crear un grafo por cada lugar

In [11]:
amigosGeneral = []

# Cargar Nodos y Mac_usr

Para facilitar la creacion de los grafos se realizo una especie de hash a cada mac_usr, para que sea un numero entero que la ligue al valor real. Se encuentra en el archivo de Nodos.

In [12]:
nodos = pd.DataFrame.from_csv("./Nodos/nodos.csv", sep = ",")
nodos.head()

,Mac
id,
1,50:1A:C5:EE:01:07
2,A0:E4:53:EC:30:77
3,94:35:0A:AB:C1:81
4,F4:09:D8:6E:D6:47
5,0C:B3:19:0A:FB:FD


# Aeropuerto 

## Top 10 Arcos con mayor Peso

Anteriormente se crearon archivos .csv para cada lugar, el cual se compone de Mac1, Mac2, Peso, por conveniencia se encuentran ordenado en base a su Peso.

In [13]:
aeropuerto_arcos = pd.DataFrame.from_csv("aeropuertoPeso.csv", sep = ",").reset_index()
top10_aeropuerto = aeropuerto_arcos.head(10)
top10_aeropuerto

,Mac1,Mac2,peso
0,426,638,47
1,426,149,31
2,697,113,30
3,113,426,29
4,149,320,29
5,61079,426,28
6,697,426,24
7,813,426,24
8,254,426,23
9,61079,113,22


Como se puede observar no tenemos las mac_usr solo tenemos sus ids por lo tanto mergearemos esto con el dataframe de nodos.

In [14]:
top10_aeropuerto = pd.merge(top10_aeropuerto, nodos.reset_index().rename(columns={'id': 'id', 'Mac': 'Mac_usr1'}), left_on = "Mac1", right_on ="id")
top10_aeropuerto = pd.merge(top10_aeropuerto, nodos.reset_index().rename(columns={'id': 'id', 'Mac': 'Mac_usr2'}), left_on = "Mac2", right_on ="id")
top10_aeropuerto = top10_aeropuerto[["Mac1", "Mac_usr1", "Mac2", "Mac_usr2", "peso"]]
top10_aeropuerto

,Mac1,Mac_usr1,Mac2,Mac_usr2,peso
0,426,2C:D0:5A:72:6D:AA,638,90:94:E4:0A:4B:E6,47
1,426,2C:D0:5A:72:6D:AA,149,D0:92:9E:0F:27:14,31
2,697,18:3A:2D:FC:19:96,113,1C:B0:94:75:B9:A9,30
3,61079,00:0C:E7:80:B9:97,113,1C:B0:94:75:B9:A9,22
4,697,18:3A:2D:FC:19:96,426,2C:D0:5A:72:6D:AA,24
5,113,1C:B0:94:75:B9:A9,426,2C:D0:5A:72:6D:AA,29
6,61079,00:0C:E7:80:B9:97,426,2C:D0:5A:72:6D:AA,28
7,813,24:E3:14:85:4D:79,426,2C:D0:5A:72:6D:AA,24
8,254,2C:AE:2B:34:07:0F,426,2C:D0:5A:72:6D:AA,23
9,149,D0:92:9E:0F:27:14,320,D8:EB:97:23:BE:30,29


## Crear Grafo

In [15]:
conexiones_file = open("aeropuertoPeso.csv", 'rb')
conexiones_file.readline()
aeropuerto_conexiones = nx.read_edgelist(conexiones_file, delimiter=',', nodetype=int, data=(('weight',int),))
conexiones_file.close()
"Grafo aeropuerto Creado"

'Grafo aeropuerto Creado'

## Analisis Top 10 Usuarios con mayor grado(numero de arcos)

### Calcular el degree de cada nodo

In [16]:
degrees_aeropuerto= aeropuerto_conexiones.degree()

### Ordenar en base al grado el dic

In [17]:
items_aeropuerto = list(degrees_aeropuerto.items())
items_aeropuerto.sort(key=lambda x: x[1], reverse=True)
items_aeropuerto[:5]

[(813, 7132), (426, 6218), (638, 5938), (320, 5885), (113, 5148)]

### Recorrer y analizar cada usuario del top 5

In [18]:
def obtenerAnalisisPesos(usuario, dataframe):
    dataFiltrada = dataframe[np.logical_or(dataframe["Mac1"] == usuario, dataframe["Mac2"] == usuario)]
    aux = list(dataFiltrada["peso"])
    return max(aux), min(aux), sum(aux)/len(aux),dataFiltrada["peso"].value_counts()
    

In [19]:
def obtenerTop5Amigos(usuario, dataframe, n):
    amigos = []
    aux1 = dataframe[dataframe["Mac1"] == usuario]
    usuarios = list(aux1["Mac2"])
    pesos = list(aux1["peso"])
    amigos += [(usuarios[i], pesos[i]) for i in range(len(usuarios))]
    aux2 = dataframe[dataframe["Mac2"] == usuario]
    usuarios = list(aux2["Mac2"])
    pesos = list(aux2["peso"])
    amigos += [(usuarios[i], pesos[i]) for i in range(len(usuarios))]
    amigos.sort(key=lambda x: x[1], reverse=True)
    return amigos[:n]
    
    

In [20]:
def analizarConexiones(usuario, dataframe):
    return dataframe[dataframe["mac_usr"] == usuario]

In [25]:
print("ANALISIS TOP USUARIOS AEROPUERTO")
for mac_id,degree in items_aeropuerto[:5]:
    mac_usr = nodos["Mac"][mac_id]
    print("\n\nAnalisis del Usuario", mac_usr)
    print("  + Degree:", degree)
    maxP, minP, avgP, value = obtenerAnalisisPesos(mac_id, aeropuerto_arcos)
    print("  + Maximo Peso:",maxP)
    print("  + Minimo Peso:",minP)
    print("  + Peso Promedio:",avgP)
    print("\n  + Analisis Pesos:")
    value = value.reset_index()
    aux1= list(value["index"])
    aux2= list(value["peso"])
    print("     Peso    #NumVeces")
    for i in range(len(aux1)):
        print("     ",str(aux1[i]).ljust(3, ' '), "    ", aux2[i])
    print("\n  + Analisis Top10 usuarios con el que más se conecta")
    amigos = obtenerTop5Amigos(mac_id, aeropuerto_arcos, 10)
    print(amigos)
    for amigo, peso in amigos:
        pareja = [mac_id, amigo]
        pareja.sort()
        amigosGeneral += (pareja, peso)
    print("\n  + Analisis Conexiones del Usuario:")
    datosUsuario = datos[datos["mac_usr"] == mac_usr]
    print(datosUsuario.head())
    #print(" + Degree: ",degree)
    #print(" + Max Peso: ",)

ANALISIS TOP USUARIOS AEROPUERTO


Analisis del Usuario 24:E3:14:85:4D:79
  + Degree: 7132
  + Maximo Peso: 24
  + Minimo Peso: 1
  + Peso Promedio: 1.20064498037

  + Analisis Pesos:
     Peso    #NumVeces
      1        6342
      2        583
      3        100
      4        37
      5        20
      7        14
      6        8
      10       5
      8        4
      11       3
      15       3
      9        3
      12       2
      19       2
      24       1
      20       1
      16       1
      21       1
      14       1
      17       1

  + Analisis Top10 usuarios con el que más se conecta
[(426, 24), (320, 21), (149, 20), (254, 19), (638, 19), (813, 17), (271, 16), (6840, 15), (582, 15), (291, 15)]

  + Analisis Conexiones del Usuario:
                  datetime  sitio_id            mac_usr      so navegador  \
470208 2016-07-09 16:25:26        34  24:E3:14:85:4D:79  iPhone     OTROS   
503675 2016-04-26 23:49:20        48  24:E3:14:85:4D:79  iPhone     OTROS   
508131 